_한국어로 기계번역됨_


# 미리 구축된 ReAct 에이전트에 인간-상호작용 프로세스 추가하는 방법

<div class="admonition tip">
    <p class="admonition-title">전제 조건</p>
    <p>
        이 가이드는 다음 사항에 대한 지식을 전제로 합니다:
        <ul>
            <li>            
                <a href="https://langchain-ai.github.io/langgraph/concepts/human_in_the_loop/">
                    인간-상호작용
                </a>
            </li>
            <li>
                <a href="https://langchain-ai.github.io/langgraph/concepts/agentic_concepts/">
                    에이전트 아키텍처
                </a>                   
            </li>
            <li>
                <a href="https://python.langchain.com/docs/concepts/chat_models/">
                    채팅 모델
                </a>
            </li>
            <li>
                <a href="https://python.langchain.com/docs/concepts/tools/">
                    도구
                </a>
            </li>            
        </ul>
    </p>
</div> 

이 가이드는 미리 구축된 ReAct 에이전트에 인간-상호작용 프로세스를 추가하는 방법을 보여줍니다. 미리 구축된 ReAct 에이전트를 시작하는 방법에 대해서는 [이 튜토리얼](../create-react-agent)을 참조하세요.

도구가 호출되기 전에 중단점을 추가하려면 `create_react_agent`에 `interrupt_before=["tools"]`를 전달하면 됩니다. 이를 사용하기 위해서는 체크포인터를 사용해야 한다는 점에 유의하세요.


## 설정

먼저 필요한 패키지를 설치하고 API 키를 설정합시다.


In [2]:
%%capture --no-stderr
%pip install -U langgraph langchain-openai


In [3]:
import getpass
import os


def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")


_set_env("OPENAI_API_KEY")


<div class="admonition tip">
    <p class="admonition-title">LangGraph 개발을 위해 <a href="https://smith.langchain.com">LangSmith</a> 설정하기</p>
    <p style="padding-top: 5px;">
        LangSmith에 가입하여 LangGraph 프로젝트의 문제를 신속하게 파악하고 성능을 개선하세요. LangSmith를 사용하면 LangGraph로 구축한 LLM 앱의 디버그, 테스트 및 모니터링을 위해 추적 데이터를 사용할 수 있습니다 — 시작하는 방법에 대한 자세한 내용은 <a href="https://docs.smith.langchain.com">여기</a>에서 확인하세요. 
    </p>
</div>


## 코드


In [4]:
# First we initialize the model we want to use.
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o", temperature=0)


# For this tutorial we will use custom tool that returns pre-defined values for weather in two cities (NYC & SF)
from typing import Literal

from langchain_core.tools import tool


@tool
def get_weather(location: str):
    """Use this to get weather information from a given location."""
    if location.lower() in ["nyc", "new york"]:
        return "It might be cloudy in nyc"
    elif location.lower() in ["sf", "san francisco"]:
        return "It's always sunny in sf"
    else:
        raise AssertionError("Unknown Location")


tools = [get_weather]

# We need a checkpointer to enable human-in-the-loop patterns
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

# Define the graph

from langgraph.prebuilt import create_react_agent

graph = create_react_agent(
    model, tools=tools, interrupt_before=["tools"], checkpointer=memory
)


## 사용법


In [7]:
def print_stream(stream):
    """A utility to pretty print the stream."""
    for s in stream:
        message = s["messages"][-1]
        if isinstance(message, tuple):
            print(message)
        else:
            message.pretty_print()


In [8]:
from langchain_core.messages import HumanMessage

config = {"configurable": {"thread_id": "42"}}
inputs = {"messages": [("user", "what is the weather in SF, CA?")]}

print_stream(graph.stream(inputs, config, stream_mode="values"))


================================ Human Message =================================

what is the weather in SF, CA?
================================== Ai Message ==================================
Tool Calls:
  get_weather (call_YjOKDkgMGgUZUpKIasYk1AdK)
 Call ID: call_YjOKDkgMGgUZUpKIasYk1AdK
  Args:
    location: SF, CA


우리는 그래프가 올바른 위치에서 멈췄음을 확인할 수 있습니다:


In [9]:
snapshot = graph.get_state(config)
print("Next step: ", snapshot.next)


Next step:  ('tools',)


이제 우리는 다음 노드로 진행하기 전에 도구 호출을 승인하거나 편집할 수 있습니다. 도구 호출을 승인하고 싶다면, `None` 입력으로 그래프 스트리밍을 계속하면 됩니다. 도구 호출을 편집하고 싶다면, 상태를 업데이트하여 올바른 도구 호출을 설정해야 하며, 업데이트가 적용된 후 계속 진행할 수 있습니다.

우리는 재개해 볼 수 있으며, 오류가 발생하는 것을 볼 수 있습니다:


In [10]:
print_stream(graph.stream(None, config, stream_mode="values"))


================================== Ai Message ==================================
Tool Calls:
  get_weather (call_YjOKDkgMGgUZUpKIasYk1AdK)
 Call ID: call_YjOKDkgMGgUZUpKIasYk1AdK
  Args:
    location: SF, CA
================================= Tool Message =================================
Name: get_weather

Error: AssertionError('Unknown Location')
 Please fix your mistakes.
================================== Ai Message ==================================
Tool Calls:
  get_weather (call_CLu9ofeBhtWF2oheBspxXkfE)
 Call ID: call_CLu9ofeBhtWF2oheBspxXkfE
  Args:
    location: San Francisco, CA


이 오류는 "샌프란시스코, CA"라는 도구 인수가 우리 도구가 인식하지 못하는 위치이기 때문에 발생했습니다.

우리 도구가 "샌프란시스코, CA"를 알 수 없는 위치로 처리하기 때문에 "샌프란시스코"를 검색하도록 도구 호출을 수정하는 방법을 보여주겠습니다. 주를 업데이트한 후 그래프 스트리밍을 다시 시작하면 오류가 발생하지 않아야 합니다.


In [11]:
state = graph.get_state(config)

last_message = state.values["messages"][-1]
last_message.tool_calls[0]["args"] = {"location": "San Francisco"}

graph.update_state(config, {"messages": [last_message]})


{'configurable': {'thread_id': '42',
  'checkpoint_ns': '',
  'checkpoint_id': '1ef801d1-5b93-6bb9-8004-a088af1f9cec'}}

In [12]:
print_stream(graph.stream(None, config, stream_mode="values"))


================================== Ai Message ==================================
Tool Calls:
  get_weather (call_CLu9ofeBhtWF2oheBspxXkfE)
 Call ID: call_CLu9ofeBhtWF2oheBspxXkfE
  Args:
    location: San Francisco
================================= Tool Message =================================
Name: get_weather

It's always sunny in sf
================================== Ai Message ==================================

The weather in San Francisco is currently sunny.


환상적입니다! 우리의 그래프가 샌프란시스코의 날씨를 쿼리하는 데 적절하게 업데이트되었고, 도구로부터 "샌프란시스코는 항상 맑습니다"라는 올바른 응답을 받았으며, 그에 따라 사용자에게 응답했습니다.
